# Nimekahtluste tuvastamine (NER)

See märkmik pärineb [AI algajatele mõeldud õppekavast](http://aka.ms/ai-beginners).

Selles näites õpime, kuidas treenida NER-mudelit [Annoteeritud korpus nimekahtluste tuvastamiseks](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) andmestiku abil Kaggle'ist. Enne jätkamist laadige palun alla [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) fail praegusesse kataloogi.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Andmehulga ettevalmistamine

Alustame andmehulga lugemisega andmeraamistikku. Kui soovid rohkem teada saada Pandase kasutamise kohta, külasta [andmetöötluse õppetundi](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) meie [Algajate andmeteaduse juhendis](http://aka.ms/datascience-beginners)


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Leiame unikaalsed sildid ja loome otsingusõnastikud, mida saame kasutada siltide teisendamiseks klassinumbriteks:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Nüüd peame sama tegema sõnavaraga. Lihtsuse huvides loome sõnavara, arvestamata sõnade sagedust; päriselus võiksite kasutada Keras vektoriseerijat ja piirata sõnade arvu.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Me peame looma lauseandmestiku treenimiseks. Läheme läbi algse andmestiku ja eraldame kõik üksikud laused `X` (sõnade loendid) ja `Y` (tokenite loendid):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


Nüüd vektoritame kõik sõnad ja tokenid:


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Lihtsuse huvides täidame kõik laused 0 tokenitega maksimaalse pikkuseni. Tegelikkuses võiksime kasutada nutikamat strateegiat ja täita järjestusi ainult ühe minibatchi piires.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Tokenite klassifitseerimise võrgu määratlemine

Kasutame tokenite klassifitseerimiseks kahekihilist kahesuunalist LSTM-võrku. Selleks, et rakendada tihedat klassifikaatorit iga viimase LSTM-kihi väljundi jaoks, kasutame `TimeDistributed` konstruktsiooni, mis kordab sama tihedat kihti iga LSTM-i väljundi jaoks igal sammul:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Pange tähele, et me määrame siin selgesõnaliselt `maxlen` meie andmekogumi jaoks - kui soovime, et võrk suudaks töötada muutuva pikkusega järjestustega, peame võrgu määratlemisel olema veidi nutikamad.

Nüüd treenime mudelit. Kiiruse huvides treenime ainult ühe epohhi, kuid võite proovida treenida pikema aja jooksul. Samuti võiksite eraldada osa andmekogumist treeningandmeteks, et jälgida valideerimise täpsust.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Tulemuse testimine

Vaatame nüüd, kuidas meie entiteedi tuvastamise mudel töötab näidislausega:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Peamine mõte

Isegi lihtne LSTM-mudel annab NER-i puhul mõistlikke tulemusi. Kuid palju paremate tulemuste saavutamiseks tasub kasutada suuri eeltreenitud keelemudeleid, nagu BERT. BERT-i treenimist NER-i jaoks Huggingface Transformers teegiga on kirjeldatud [siin](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Lahtiütlus**:  
See dokument on tõlgitud, kasutades AI tõlketeenust [Co-op Translator](https://github.com/Azure/co-op-translator). Kuigi püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algkeeles tuleks lugeda autoriteetseks allikaks. Olulise teabe puhul on soovitatav kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
